In [ ]:
from datetime import datetime

# channel anysis
# https://www.youtube.com/watch?v=8jsFGGPTblE


# indiadrew channel analysis


channel_id = "UCZclhynkanjPVTY12fEmj3A"


from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime

# Replace 'YOUR_API_KEY' with your actual API key
api_key = "your api"
youtube = build('youtube', 'v3', developerKey=api_key)

# Replace 'CHANNEL_ID' with the ID of the desired YouTube channel
# channel_id = 'UCdyQheT22ozdTbo8nUD5gBA'

# Fetch all videos from the channel's uploads playlist
def get_all_videos_from_playlist(playlist_id):
    video_id_list = []
    title_list = []
    description_list = []
    published_date_list = []

    next_page_token = None
    while True:
        playlist_items = youtube.playlistItems().list(
            part='snippet,contentDetails',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in playlist_items['items']:
            video_id = item['contentDetails']['videoId']
            video_id_list.append(video_id)
            title_list.append(item['snippet']['title'])
            description_list.append(item['snippet']['description'])
            published_date_list.append(item['snippet']['publishedAt'])

        next_page_token = playlist_items.get('nextPageToken')
        if not next_page_token:
            break

    return pd.DataFrame({
        'video_id': video_id_list,
        'title': title_list,
        'description': description_list,
        'published_date': published_date_list
    })

# Get video statistics (like count, comment count, etc.)
def get_video_statistics(video_ids):
    video_id_list = []
    like_count_list = []
    comment_count_list = []
    view_count_list = []

    for i in range(0, len(video_ids), 50):
        video_ids_batch = video_ids[i:i + 50]
        video_response = youtube.videos().list(
            part='statistics',
            id=','.join(video_ids_batch)
        ).execute()

        for item in video_response['items']:
            video_id_list.append(item['id'])
            like_count_list.append(item['statistics'].get('likeCount', 0))
            comment_count_list.append(item['statistics'].get('commentCount', 0))
            view_count_list.append(item['statistics'].get('viewCount', 0))

    return pd.DataFrame({
        'video_id': video_id_list,
        'like_count': like_count_list,
        'comment_count': comment_count_list,
        'view_count': view_count_list
    })

# Combine video details and statistics
def fetch_youtube_data(channel_id):
    playlist_id = 'UU' + channel_id[2:]  # Convert to uploads playlist ID
    video_details_df = get_all_videos_from_playlist(playlist_id)
    video_stats_df = get_video_statistics(video_details_df['video_id'].tolist())
    
    # Merge video details with statistics
    final_df = pd.merge(video_details_df, video_stats_df, on='video_id')
    return final_df

# Fetch data for the channel
youtube_data_df = fetch_youtube_data(channel_id)

# Save to CSV
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
youtube_data_df.to_csv(f"F:\\selenium_youtube_data\\{now}_youtube_data_channel_{channel_id}.csv", index=False)

# Display the dataframe
youtube_data_df





pd.set_option('display.max_colwidth', None)

new_df=youtube_data_df
new_df

new_df['published_date'] = pd.to_datetime(new_df['published_date']).dt.date


new_df['comment_count'] = pd.to_numeric(new_df['comment_count'], errors='coerce')


new_df.to_excel("new_my_youtube.xlsx")

new_df.sort_values("like_count", ascending=False, inplace=True)
new_df